# **Prepare libraries**

In [14]:
!pip install -U datasets
!pip install -qU transformers accelerate bitsandbytes peft trl

In [32]:
from datasets import load_dataset
import torch
import pandas as pd
import numpy as np
import warnings
import json
import time

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)

from peft import get_peft_model, LoraConfig
from trl import SFTTrainer

from huggingface_hub import notebook_login

# **Load dataset**

In [16]:
dataset = load_dataset("hoskinson-center/proofnet")

print(dataset)

dataset['test'][0]

DatasetDict({
    test: Dataset({
        features: ['id', 'nl_statement', 'nl_proof', 'formal_statement', 'src_header'],
        num_rows: 186
    })
    validation: Dataset({
        features: ['id', 'nl_statement', 'nl_proof', 'formal_statement', 'src_header'],
        num_rows: 185
    })
})


{'id': 'Rudin|exercise_1_1b',
 'nl_statement': 'If $r$ is rational $(r \\neq 0)$ and $x$ is irrational, prove that $rx$ is irrational.',
 'nl_proof': '\\begin{proof}\n\n    If $r x$ were rational, then $x=\\frac{r x}{r}$ would also be rational.\n\n\\end{proof}',
 'formal_statement': 'theorem exercise_1_1b\n(x : ℝ)\n(y : ℚ)\n(h : y ≠ 0)\n: ( irrational x ) -> irrational ( x * y ) :=',
 'src_header': 'import .common\n\nopen real complex\nopen topological_space\nopen filter\nopen_locale real \nopen_locale topology\nopen_locale big_operators\nopen_locale complex_conjugate\nopen_locale filter\n\n\nnoncomputable theory\n\n'}

In [17]:
def format_instruction(example):
    text = f"""<start_of_turn>user\n{example["nl_statement"]}<end_of_turn>\n<start_of_turn>model\n{example['nl_proof']}<end_of_turn>"""

    return {'prompt': text}

dataset = dataset.map(format_instruction)

In [18]:
dataset['test'][0]

{'id': 'Rudin|exercise_1_1b',
 'nl_statement': 'If $r$ is rational $(r \\neq 0)$ and $x$ is irrational, prove that $rx$ is irrational.',
 'nl_proof': '\\begin{proof}\n\n    If $r x$ were rational, then $x=\\frac{r x}{r}$ would also be rational.\n\n\\end{proof}',
 'formal_statement': 'theorem exercise_1_1b\n(x : ℝ)\n(y : ℚ)\n(h : y ≠ 0)\n: ( irrational x ) -> irrational ( x * y ) :=',
 'src_header': 'import .common\n\nopen real complex\nopen topological_space\nopen filter\nopen_locale real \nopen_locale topology\nopen_locale big_operators\nopen_locale complex_conjugate\nopen_locale filter\n\n\nnoncomputable theory\n\n',
 'prompt': '<start_of_turn>user\nIf $r$ is rational $(r \\neq 0)$ and $x$ is irrational, prove that $rx$ is irrational.<end_of_turn>\n<start_of_turn>model\n\\begin{proof}\n\n    If $r x$ were rational, then $x=\\frac{r x}{r}$ would also be rational.\n\n\\end{proof}<end_of_turn>'}

In [19]:
notebook_login()

# Laod Gemma-2-2b

In [24]:
model_id = "google/gemma-2-2b"

bnb_config = BitsAndBytesConfig(load_in_8bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)


model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [25]:
tokenizer.pad_token = tokenizer.eos_token

In [26]:
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
dataset = dataset['test'].train_test_split(test_size=0.1)
dataset

Map:   0%|          | 0/167 [00:00<?, ? examples/s]

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'nl_statement', 'nl_proof', 'formal_statement', 'src_header', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 17
    })
    test: Dataset({
        features: ['id', 'nl_statement', 'nl_proof', 'formal_statement', 'src_header', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 2
    })
})

In [27]:
train_data = dataset["train"]
test_data = dataset["test"]

In [28]:
print(train_data[0])

{'id': 'Putnam|exercise_2001_a5', 'nl_statement': 'Prove that there are unique positive integers $a, n$ such that $a^{n+1}-(a+1)^n=2001$.', 'nl_proof': '\\begin{proof}\n\n    Suppose $a^{n+1} - (a+1)^n = 2001$.\n\nNotice that $a^{n+1} + [(a+1)^n - 1]$ is a multiple of $a$; thus\n\n$a$ divides $2002 = 2 \\times 7 \\times 11 \\times 13$.\n\n\n\nSince $2001$ is divisible by 3, we must have $a \\equiv 1 \\pmod{3}$,\n\notherwise one of $a^{n+1}$ and $(a+1)^n$ is a multiple of 3 and the\n\nother is not, so their difference cannot be divisible by 3. Now\n\n$a^{n+1} \\equiv 1 \\pmod{3}$, so we must have $(a+1)^n \\equiv 1\n\n\\pmod{3}$, which forces $n$ to be even, and in particular at least 2.\n\n\n\nIf $a$ is even, then $a^{n+1} - (a+1)^n \\equiv -(a+1)^n \\pmod{4}$.\n\nSince $n$ is even, $-(a+1)^n \\equiv -1 \\pmod{4}$. Since $2001 \\equiv 1\n\n\\pmod{4}$, this is impossible. Thus $a$ is odd, and so must divide\n\n$1001 = 7 \\times 11 \\times 13$. Moreover, $a^{n+1} - (a+1)^n \\equiv a\n\n\

In [29]:
def get_completion(query: str, model, tokenizer):

  prompt_template = """<start_of_turn>user
  {query}
  <end_of_turn>
  <start_of_turn>model
  <end_of_turn>
  """
  prompt = prompt_template.format(query=query)
  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encodeds.to("cuda:0")
  generated_ids = model.generate(**model_inputs, max_new_tokens=256)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return decoded


result = get_completion(query=train_data[0]['nl_statement'], model=model, tokenizer=tokenizer)
print(result)

user
  Prove that there are unique positive integers $a, n$ such that $a^{n+1}-(a+1)^n=2001$.
  
  model
  
    StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  StoryboardSegue
  Storyb

In [33]:

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",]
)

model = get_peft_model(model, lora_config)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        num_train_epochs=2,
        max_steps=200,
        logging_steps=1,
        warmup_steps=10,
        logging_strategy="steps",
        learning_rate=2e-5,
        fp16=False,
        bf16=False,
        group_by_length=True,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due 

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,10.635100
2,11.282100
3,10.519700
4,12.283600
5,10.759200
6,11.242000
7,11.137500
8,11.296800
9,9.759400
10,10.211300


TrainOutput(global_step=200, training_loss=3.002581617832184, metrics={'train_runtime': 1078.9926, 'train_samples_per_second': 1.483, 'train_steps_per_second': 0.185, 'total_flos': 8529894981563904.0, 'train_loss': 3.002581617832184, 'epoch': 94.11764705882354})

In [34]:
result = get_completion(query="Prove that every compact metric space $K$ has a countable base.",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)

user
  Prove that every compact metric space $K$ has a countable base.
  
  model
  
  \begin{proof}

    First off, let us assume that $K$ is a compact metric space. Therefore, every subset of $K$ is bounded. Additionally, we can assume that $K$ is complete. Therefore, each subset of $K$ is sequentially closed. Hence, $K$ is metrizable.

Now, we can use the following result:

\begin{proposition}

    Every metrizable space is regular.

\end{proposition}

Therefore, $K$ is regular. Consequently, $K$ has a base $\mathcal{B}$. Now, we can define a recursive sequence of subsets of $\mathcal{B}$ as follows:

$$

A_0 = \emptyset, \quad A_{n+1} = \left\{B \in \mathcal{B} \mid B \cap\left(A_n \cup\{K\}\right) \text { is at most countable }\right\}

$$

We claim that $\left\{A_n\right\}$ is a decreasing sequence whose limit is equal to $K$. In fact, $K \cap A_{n+1} \subset K \cap A_n \cup\{K\}$ is closed and bounded


In [35]:
result = get_completion(query="Prove that every compact metric space $K$ is closed",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)

user
  Prove that every compact metric space $K$ is closed
  
  model
  
  \begin{proof}

    First off, let us assume that $K$ is a compact metric space. Therefore, $K$ is Hausdorff, regular, and normal.

In a metric space, a closed set is equal to its complement. Therefore, we can represent $K$ as $K = \mathrm{cl}(K)^\mathrm{c}$.

In a Hausdorff space, a closed subspace is closed. Therefore, $\mathrm{cl}(K)$ is closed in the Euclidean plane $\mathbb{R}^2$.

In a regular space, a closed subspace is closed. Therefore, $K^\mathrm{c}$ is closed in $\mathbb{R}^2$.

In a normal space, the closed subspace intersection property holds. Therefore, $\mathrm{cl}(K) \cap K^\mathrm{c} = \varnothing$, which says that $K$ is closed in the Euclidean plane $\mathbb{R}^2$.

\end{proof}

model

\begin{proof}

    First off, let us assume that $K$ is a compact metric space. Therefore, $K$ is Hausdorff, regular, and normal.

In a metric space, a closed set is equal


In [ ]:
trainer.save_model("outputs")
tokenizer.save_pretrained("outputs")